# VISUALIZE OUTPUT FROM REGISTRATION

- change `DATA_PTH` to point to your main processing output folder
- change `REG_DIR` to point to the directory where registration output is stored. tifs should be stored there as `0000.tif`

`conda activate imgpro`

## LOAD DATA

In [ ]:
import os
import tifffile
import numpy as np
import importlib
import utils

import matplotlib.pyplot as plt
import scipy.ndimage as ndi

from microfilm.microplot import microshow
from microfilm.microplot import Micropanel
from microfilm.microanim import Microanim
from microfilm.microanim import Microanimpanel
from microfilm.microplot import Microimage

# DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\fluorescent_beads_ch_align\20251219'
DATA_PTH = r'D:\DATA\g5ht-free\20260123'

INPUT_ND2 = 'date-20260123_strain-ISg5HT-nsIS180_condition-fedpatch_worm005.nd2'

INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)

NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\noise\noise_042925.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41

noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)
num_frames, height, width, num_channels = utils.get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 
beads_alignment_file = utils.get_beads_alignment_file(INPUT_ND2_PTH)

print(INPUT_ND2)
print('Num z-slices: ', STACK_LENGTH)
print('Number of frames: ', num_frames)
print('Height: ', height)
print('width: ', width)
print('Number of channels: ', num_channels)
print('Beads alignment file: ', beads_alignment_file)

In [ ]:
z_to_plot = np.arange(0, STACK_LENGTH-2, 2)
z_to_plot.shape

In [ ]:
input_dir = os.path.splitext(INPUT_ND2_PTH)[0]
output_path = os.path.join(input_dir, 'registered_fixed_sweep')
warped_path = os.path.join(input_dir, 'warped')

plt.close('all')

fixed_list = np.arange(0, 400, 50)
mov_list = np.arange(0, 400, 30)

# fixed_list = np.arange(0, 150, 50)
# mov_list = np.arange(0, 150, 30)

z_to_plot = np.arange(0, STACK_LENGTH-2, 2)

nrows = 5
ncols = 4

fig_sc = 3
dpi = 72

for fixed in fixed_list:
    
    # load fixed stack
    fixed_file_path = os.path.join(warped_path,f'{fixed:04d}.tif')
    fixed_stack = tifffile.imread(fixed_file_path)[:,1,:,:]
    
    # use Micropanel to plot zslices for fixed stack
    panel = Micropanel(rows=nrows, cols=ncols, figscaling=fig_sc, dpi=dpi)
    for iz,z in enumerate(z_to_plot):
        microim1 = Microimage(fixed_stack[np.newaxis,z,:,:], cmaps='magenta', fig_scaling=fig_sc, dpi=dpi, channel_names='fixed')
        panel.add_element([iz//ncols,iz%ncols], microim1)
        # panel.add_channel_label()
    
    for mov in mov_list:
        # load stacks
        output_file_path = os.path.join(output_path, f'fixed_{fixed:04d}_mov_{mov:04d}.tif')
        output_stack = tifffile.imread(output_file_path)[:,1,:,:]
        
        # use Micropanel to plot zslices for fixed stack
        panel = Micropanel(rows=nrows, cols=ncols, figscaling=fig_sc, dpi=dpi)
        for iz,z in enumerate(z_to_plot):
            microim1 = Microimage(output_stack[np.newaxis,z,:,:], cmaps='cyan', fig_scaling=fig_sc, dpi=dpi, channel_names='moving')
            panel.add_element([iz//ncols,iz%ncols], microim1)
            # panel.add_channel_label()
        
    #     break
    # break


In [ ]:
input_dir = os.path.splitext(INPUT_ND2_PTH)[0]
output_path = os.path.join(input_dir, 'registered_fixed_sweep')
warped_path = os.path.join(input_dir, 'warped')
save_path = os.path.join(input_dir, 'registered_fixed_sweep_pngs')
os.makedirs(save_path, exist_ok=True)

plt.close('all')

fixed_list = np.arange(0, 400, 50)
mov_list = np.arange(0, 400, 30)

# fixed_list = np.arange(0, 150, 50)
# mov_list = np.arange(0, 150, 30)

z_to_plot = np.arange(0, STACK_LENGTH-2, 2)

nrows = 5
ncols = 4

fig_sc = 4
dpi = 300

for fixed in fixed_list:
    
    # load fixed stack
    fixed_file_path = os.path.join(warped_path,f'{fixed:04d}.tif')
    fixed_stack = tifffile.imread(fixed_file_path)[:,1,:,:]
    
    # # use Micropanel to plot zslices for fixed stack
    # panel = Micropanel(rows=nrows, cols=ncols, figscaling=fig_sc, dpi=dpi)
    # for iz,z in enumerate(z_to_plot):
    #     microim1 = Microimage(fixed_stack[np.newaxis,z,:,:], cmaps='magenta', fig_scaling=fig_sc, dpi=dpi, channel_names='fixed')
    #     panel.add_element([iz//ncols,iz%ncols], microim1)
    #     # panel.add_channel_label()
    
    for mov in mov_list:
        # load stacks
        output_file_path = os.path.join(output_path, f'fixed_{fixed:04d}_mov_{mov:04d}.tif')
        output_stack = tifffile.imread(output_file_path)[:,1,:,:]
        
        cat_stack = np.concatenate((fixed_stack[np.newaxis,:,:,:], output_stack[np.newaxis,:,:,:]), axis=0)
        
        # use Micropanel to plot zslices for fixed stack
        panel = Micropanel(rows=nrows, cols=ncols, figscaling=fig_sc, dpi=dpi)
        for iz,z in enumerate(z_to_plot):
            if iz==0:
                label_text=f'fixed {fixed:04d}', f'moving {mov:04d}'
            else:
                label_text=None
            microim1 = Microimage(cat_stack[:,z,:,:], cmaps=['magenta', 'cyan'], fig_scaling=fig_sc, dpi=dpi, channel_names=['fixed', 'moving'],
                                  label_text=label_text)
            panel.add_element([iz//ncols,iz%ncols], microim1)
            plt.savefig(os.path.join(save_path, f'fixed_{fixed:04d}_mov_{mov:04d}_slices.png'), dpi=dpi)
            # panel.add_channel_label()
        
    #     break
    # break


More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
